# 1. 모델 YOLOV5 다운로드

In [ ]:
%cd /content
!git clone https://github.com/ultralytics/yolov5 
%cd yolov5
%pip install -qr requirements.txt

import torch
from yolov5 import utils
display = utils.notebook_init()  # checks

# 2. 추론(Inference)

### detect.py- yolov5 모델을 이용한 추론

### 주어진 이미지를 yolov5에서 제공되는 yolov5s.pt로 detect 해보자.

In [ ]:
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images/

## 잘 detect 되었는지 확인해보자.

- detect된 이미지 화일명을 출력

In [ ]:
!ls runs/detect/exp

- detect된 이미지를 출력

In [ ]:
display.Image(filename='runs/detect/exp/bus.jpg', width=600)

# 3. 포트홀 탐지 모델

### 포트홀 데이터셋 다운로드(roboflow)

포트홀 데이터셋: https://public.roboflow.com/object-detection/pothole

- 다운로드할 디렉토리를 만들고 그쪽으로 이동한다.

## 포트홀 데이터셋을 설치한다.

In [ ]:
%mkdir /content/pothole
%cd /content/pothole
!curl -L "https://public.roboflow.com/ds/m0E5Tx4om4?key=kEKT9jhfJu" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

## pothole에 무엇이 있는지 확인해보자!

In [ ]:
!ls /content/pothole

## list를 정리(train, valid의 이미지 갯수 확인)

In [ ]:
from glob import glob

train_img_list=glob('/content/pothole/train/images/*.jpg')
valid_img_list=glob('/content/pothole/valid/images/*.jpg')
print(len(train_img_list), len(valid_img_list))

## 현재 data.yaml화일을 열어 보자.

In [ ]:
%cat /content/pothole/data.yaml

# pothole에서 data.yaml 화일을 열고 그 내용을 아래와 같이 수정한다.

## train: /content/pothole/train/images
## val: /content/pothole/valid/images

## nc: 1
## names: ['pothole']

## 원하는데로 저장이 되는지 확인한다.

In [ ]:
%cat /content/pothole/data.yaml

# 4. 모델 구성

### 처음에는 미리 개발된 yolov5s.yaml 이 어떻게 구성되었는지 확인한다.

In [ ]:
%cat /content/yolov5/models/yolov5s.yaml

# 5. 학습하기

## - train.py 사용

In [ ]:
%%time
%cd /content/yolov5
!python train.py --img 640 --batch 16 --epochs 5 --data /content/pothole/data.yaml --weights yolov5s.pt --cache

## train_batch0.jpg 출력

In [ ]:
display.Image(filename='/content/yolov5/runs/train/exp/train_batch0.jpg', width=1000)

# 6. 학습결과를 그래프로 확인하기

## tensorboard 로 학습이 잘 진행되었는지 확인

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

# 7. pothole_18s.mp4 동영상을 detect 해보자.

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/exp/weights/last.pt --img 640 --conf 0.5 --source /content/pothole_18s.mp4

# 8. 모델(xx.pt) 내보내기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%mkdir /content/drive/My\ Drive/pothole
%cp /content/yolov5/runs/train/exp/weights/best.pt /content/drive/My\ Drive/pothole/